In [1]:
import os
import pandas as pd
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed

# Define the directories and output path
directory = r'D:\SamsungSTF\Processed_Data\TripByTrip'
output_path = r'C:\Users\BSL\Desktop\result_files_acc.csv'

# Function to check if a file contains acceleration values >= 7 or <= -9.8
def check_file(filepath):
    try:
        df = pd.read_csv(filepath)
        if 'acceleration' in df.columns:
            condition1 = (df['acceleration'] > 7).any()
            condition2 = (df['acceleration'] < -9.8).any()
            if condition1 or condition2:
                return os.path.basename(filepath), condition1, condition2
    except Exception as e:
        print(f"Error processing file {filepath}: {e}")
    return None

def main():
    # Get list of all CSV files in the directory
    csv_files = [os.path.join(directory, filename) for filename in os.listdir(directory) if filename.endswith(".csv")]
    
    files_with_high_acceleration = []
    
    # Create a ThreadPoolExecutor
    with ThreadPoolExecutor() as executor:
        # Map the check_file function to the file paths
        futures = {executor.submit(check_file, filepath): filepath for filepath in csv_files}
        
        for future in as_completed(futures):
            result = future.result()
            if result:
                files_with_high_acceleration.append(result)
    
    # Sort the results
    files_with_high_acceleration.sort(key=lambda x: x[0])
    
    # Create DataFrame with results
    result_df = pd.DataFrame(files_with_high_acceleration, columns=['filename', 'condition1', 'condition2'])
    
    # Convert boolean values to 'O' or 'X'
    result_df['condition1'] = result_df['condition1'].map({True: 'O', False: 'X'})
    result_df['condition2'] = result_df['condition2'].map({True: 'O', False: 'X'})
    
    # Save results to a CSV file
    result_df.to_csv(output_path, index=False)
    
    return result_df

if __name__ == "__main__":
    result_files = main()
    print(result_files)

                                        filename condition1 condition2
0            bms_01241228051-2023-01-trip-14.csv          O          X
1             bms_01241228051-2023-01-trip-8.csv          O          X
2            bms_01241228051-2023-02-trip-23.csv          O          X
3            bms_01241228051-2023-02-trip-37.csv          O          X
4            bms_01241228051-2023-02-trip-39.csv          O          X
5            bms_01241228051-2023-02-trip-43.csv          O          X
6            bms_01241228051-2023-03-trip-20.csv          O          X
7            bms_01241228051-2023-03-trip-21.csv          O          X
8            bms_01241228051-2023-03-trip-26.csv          O          X
9            bms_01241228051-2023-04-trip-38.csv          O          X
10           bms_01241228051-2023-04-trip-49.csv          O          X
11           bms_01241228051-2023-05-trip-12.csv          O          X
12           bms_01241228051-2023-05-trip-15.csv          O          X
13    

In [ ]:
# from GS_plot import plot_power, plot_energy
# folder_path = os.path.join(os.path.normpath(r'D:\SamsungSTF\Processed_Data'), 'TripByTrip')
# # Load the CSV file
# file_path = r"C:\Users\BSL\Desktop\result_files_acc.csv"
# df = pd.read_csv(file_path)
# 
# # Convert the file names to a pandas Series
# file_names_series = pd.Series(df.iloc[:, 0])
# file_names_series = folder_path + '/' + file_names_series
# 
# 
# plot_power(file_names_series[:10], None, 'comparison')
# plot_energy(file_names_series[:10], None, 'comparison')
